In [ ]:
# Import of libraries

import re
import unicodedata
import itertools

import pandas as pd
import numpy as np
import pandas

# Data visualization
import seaborn as sns
import matplotlib.pylab as pl
import matplotlib as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot as plt



import requests
import io
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from bs4 import BeautifulSoup
from urllib.parse import urljoin



In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english')).union(set(stopwords.words('french')))



[nltk_data] Downloading package punkt to /home/cytech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/cytech/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/cytech/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Importation des données

In [4]:

data_1 = pd.read_csv("/home/cytech/Rag_covid/data/Kaggle/target_tables/8_risk_factors/Age.csv")

data_2 = pd.read_csv("/home/cytech/Rag_covid/data/Kaggle/target_tables/8_risk_factors/Diabetes.csv")

data_3 = pd.read_csv("/home/cytech/Rag_covid/data/Kaggle/target_tables/8_risk_factors/Overweight or obese.csv")

# Preprocessing

In [19]:
# Renommer les colonnes pour les rendre plus faciles à utiliser
data_1.columns = data_1.columns.str.strip().str.lower().str.replace(" ", "_")
data_2.columns = data_2.columns.str.strip().str.lower().str.replace(" ", "_")
data_3.columns = data_3.columns.str.strip().str.lower().str.replace(" ", "_")

data_1


,unnamed:_0,date,study,study_link,journal,severe,severe_lower_bound,severe_upper_bound,severe_p-value,severe_significant,...,fatality_adjusted,fatality_calculated,multivariate_adjustment,study_type,sample_size,study_population,added_on,critical_only,discharged_vs._death?,key_word
0,0,2020-05-29,Phenotypic characteristics and prognosis of in...,https://www.ncbi.nlm.nih.gov/pubmed/32472191/,Diabetologia,OR: 1.00,0.99,1.01,NaN,Not Significant,...,Not Adjusted,Extracted,NaN,Retrospective observational study,1317,The CORONADO study was launched in all French ...,NaN,Y,N,"[diabetes, primary, outcome, admission, age]"
1,1,2020-05-28,Obesity and COVID-19: an Italian snapshot,https://www.ncbi.nlm.nih.gov/pubmed/32463545/,Obesity,NaN,NaN,NaN,NaN,NaN,...,Not Adjusted,Extracted,NaN,Retrospective observational study,92,92 patients stayed at least one day in the COV...,NaN,N,N,"[patients, p, obesity, severe, age]"
2,2,2020-05-26,Association between Cardiovascular Burden and ...,http://medrxiv.org/cgi/content/short/2020.05.2...,MedRxiv,AOR=0.99,0.97,1.02,0.65,Not Significant,...,NaN,NaN,Age; Sex,Retrospective observational study,332,"February 5, 2020, to March 10, 2020 (followed ...",NaN,Y,N,"[patients, preprint, cvd, disease, covid]"
3,3,2020-05-26,Association between Cardiovascular Burden and ...,http://medrxiv.org/cgi/content/short/2020.05.2...,MedRxiv,AOR=0.99,0.97,1.02,0.6,Not Significant,...,NaN,NaN,Age; Sex; Chest tightness; Diabetes; Lung dise...,Retrospective observational study,332,"February 5, 2020, to March 10, 2020 (followed ...",NaN,Y,N,"[patients, preprint, cvd, disease, covid]"
4,4,2020-05-26,Association between Cardiovascular Burden and ...,http://medrxiv.org/cgi/content/short/2020.05.2...,MedRxiv,AOR=0.99,0.96,1.02,0.55,Not Significant,...,NaN,NaN,Age; Sex; Chest tightness; Diabetes; Lung dise...,Retrospective observational study,332,"February 5, 2020, to March 10, 2020 (followed ...",NaN,Y,N,"[patients, preprint, cvd, disease, covid]"
5,5,2020-05-26,Association between Cardiovascular Burden and ...,http://medrxiv.org/cgi/content/short/2020.05.2...,MedRxiv,OR=1.01,0.98,1.03,0.6,Not Significant,...,NaN,NaN,NaN,Retrospective observational study,332,"February 5, 2020, to March 10, 2020 (followed ...",NaN,Y,N,"[patients, preprint, cvd, disease, covid]"
6,6,2020-05-20,Extent of prior lung irradiation and mortality...,https://doi.org/10.1016/j.adro.2020.04.028,Adv Radiat Oncol,NaN,NaN,NaN,NaN,NaN,...,Not Adjusted,Extracted,NaN,Retrospective observational study,107,Montefiore Medical Center and Albert Einstein ...,NaN,N,N,[redirecting]
7,7,2020-05-15,Low dose of hydroxychloroquine reduces fatalit...,https://doi.org/10.1007/s11427-020-1732-2,Sci China Life Sci,NaN,NaN,NaN,NaN,NaN,...,Adjusted,Extracted,Hydroxychloroquine treated; Age; Gender; Hyper...,Retrospective observational study,550,"Tongji Hospital from Feb 1, 2020 to Apr 4, 2020",NaN,N,N,"[google, scholar, article, author, hydroxychlo..."
8,8,2020-05-15,Low dose of hydroxychloroquine reduces fatalit...,https://doi.org/10.1007/s11427-020-1732-2,Sci China Life Sci,NaN,NaN,NaN,NaN,NaN,...,Not Adjusted,Extracted,NaN,Retrospective observational study,550,"Tongji Hospital from Feb 1, 2020 to Apr 4, 2020",NaN,N,N,"[google, scholar, article, author, hydroxychlo..."
9,9,2020-05-14,Clinical Characteristics and Outcomes of Patie...,https://doi.org/10.2337/dc20-0660,Diabetes care,AOR=1.02,1.01,1.03,NaN,Significant,...,Adjusted,Extracted,Age; Albumin; Creatinine; CRP; Glucose,Retrospective observational study,904,Central Hospital of Wuhan from 1 January 2020 ...,NaN,N,Y,[]


## Création de la colonne "key_word"

In [20]:
def get_html_text(url):
    """
    Récupère le contenu texte d'une page web à partir de son URL.
    Exclut les balises script et style.
    """
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            # Supprime les balises script et style
            for element in soup(["script", "style"]):
                element.extract()
            # Récupère le texte brut
            text = soup.get_text(separator=" ")
            return text
        else:
            print(f"Erreur lors du téléchargement de la page: {url} - Code {response.status_code}")
            return ""
    except Exception as e:
        print(f"Exception lors de la lecture de la page {url}: {e}")
        return ""

In [21]:
def get_pdf_link(url):
    """
    Si l'URL ne se termine pas par .pdf, télécharge la page HTML et recherche
    le lien PDF en cherchant dans les balises <a> un href contenant 'pdf'.
    Retourne l'URL du PDF ou None s'il n'est pas trouvé.
    """
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Erreur lors du téléchargement de la page: {url} - Code {response.status_code}")
            return None
        soup = BeautifulSoup(response.text, 'html.parser')
        pdf_link = None
        # Recherche d'un lien contenant 'pdf' dans le href
        for a in soup.find_all('a', href=True):
            href = a['href']
            if 'pdf' in href.lower():
                pdf_link = urljoin(url, href)
                break
        if pdf_link is None:
            print(f"Aucun lien PDF trouvé sur la page: {url}")
        return pdf_link
    except Exception as e:
        print(f"Exception lors de l'extraction du lien PDF pour {url}: {e}")
        return None


In [22]:

def download_pdf(url):
    """
    Télécharge le contenu du PDF depuis l'URL.
    Retourne le contenu binaire du PDF ou None en cas d'erreur.
    """
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Erreur lors du téléchargement: {url} - Code {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception pour {url}: {e}")
        return None



In [23]:
def extract_text_from_pdf(pdf_content):
    """
    Extrait le texte d'un PDF à partir de son contenu binaire.
    Essaie d'abord avec PyPDF2 et, en cas d'erreur, utilise pdfminer.six.
    """
    text = ""
    try:
        pdf_file = io.BytesIO(pdf_content)
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
    except Exception as e:
        print(f"Erreur d'extraction du texte avec PyPDF2: {e}")
        # Revenir au début du fichier
        pdf_file.seek(0)
        try:
            from pdfminer.high_level import extract_text
            text = extract_text(pdf_file)
        except Exception as e2:
            print(f"Erreur d'extraction du texte avec pdfminer: {e2}")
    return text


In [24]:
def extract_html_section_text(url, container_id=None, container_class=None):
    """
    Récupère le texte d'une zone précise (spécifiée par 'container_id' ou 'container_class')
    dans une page HTML. Si la zone n'est pas trouvée, renvoie le texte du <body>.
    Supprime les balises <script> et <style> avant de renvoyer le texte brut.
    """
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Erreur lors de la requête: {url} - Code {response.status_code}")
            return ""
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # On tente de récupérer le container spécifié (priorité à l'id, sinon la classe)
        container = None
        if container_id:
            container = soup.find(id=container_id)
        elif container_class:
            container = soup.find(class_=container_class)
        
        # Si on ne trouve pas la zone ciblée, on récupère tout le body
        if not container:
            container = soup.find('body')
            if not container:
                return ""
        
        # Supprime les scripts et styles
        for element in container(["script", "style"]):
            element.extract()
        
        # Récupère le texte brut
        text = container.get_text(separator=" ")
        return text.strip()
    
    except Exception as e:
        print(f"Exception lors de l'extraction du texte depuis {url}: {e}")
        return ""


In [25]:
def extract_keywords(text, top_n=5):
    """
    Extrait les 'top_n' mots-clés du texte en se basant sur la fréquence des mots.
    """
    # Mise en minuscules et suppression de la ponctuation
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    
    # Retirer les stop words et les tokens non alphabétiques
    filtered_tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    
    # Calcul de la fréquence
    freq = {}
    for token in filtered_tokens:
        freq[token] = freq.get(token, 0) + 1
    
    # Tri par fréquence décroissante
    sorted_tokens = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    # Sélection des top_n mots-clés
    keywords = [token for token, count in sorted_tokens[:top_n]]
    return keywords

In [26]:
def process_dataframe(df, use_abstract=False):
    """
    Pour chaque ligne du DataFrame, vérifie si l'URL dans "study_link" pointe directement vers un PDF.
    Sinon, tente d'extraire le lien PDF depuis la page HTML.
    Si aucun PDF n'est trouvé, récupère le texte de la page HTML ou de la section 'abstract'
    pour en déduire 5 mots-clés (en fonction de 'use_abstract').
    Ajoute une colonne 'key_word' au DataFrame.
    """
    keywords_list = []
    for index, row in df.iterrows():
        study_url = row['study_link']
        print(f"Traitement de l'article {index} : {study_url}")
        
        # 1) Vérifie si l'URL se termine par .pdf, sinon essaie de récupérer le lien PDF
        if not study_url.lower().endswith('.pdf'):
            pdf_url = get_pdf_link(study_url)
            if pdf_url:
                # Correction pour retirer le suffixe '+html' s'il existe
                if '+html' in pdf_url:
                    pdf_url = pdf_url.replace('+html', '')
                print(f"Lien PDF trouvé: {pdf_url}")
            else:
                print("Aucun lien PDF trouvé, on va extraire le texte HTML.")
                
                # 2) Soit on cible spécifiquement l'abstract (si use_abstract=True),
                #    soit on prend tout le HTML.
                if use_abstract:
                    html_text = extract_html_section_text(study_url, container_id="eng-abstract")
                    if not html_text:  # Si pas trouvé, on essaie tout le body
                        html_text = get_html_text(study_url)
                else:
                    html_text = get_html_text(study_url)
                
                # Extraction de mots-clés
                if html_text:
                    keywords = extract_keywords(html_text, top_n=5)
                else:
                    keywords = []
                
                keywords_list.append(keywords)
                continue
        else:
            pdf_url = study_url

        # 3) Lecture du PDF depuis le web
        try:
            response = requests.get(pdf_url, timeout=10)
            if response.status_code == 200:
                pdf_content = response.content
            else:
                print(f"Erreur lors de la lecture du PDF: {pdf_url} - Code {response.status_code}")
                # En cas d'erreur, fallback sur le HTML
                if use_abstract:
                    html_text = extract_html_section_text(study_url, container_id="eng-abstract")
                    if not html_text:
                        html_text = get_html_text(study_url)
                else:
                    html_text = get_html_text(study_url)
                
                if html_text:
                    keywords = extract_keywords(html_text, top_n=5)
                else:
                    keywords = []
                keywords_list.append(keywords)
                continue
        except Exception as e:
            print(f"Exception lors de la lecture du PDF: {pdf_url}: {e}")
            if use_abstract:
                html_text = extract_html_section_text(study_url, container_id="eng-abstract")
                if not html_text:
                    html_text = get_html_text(study_url)
            else:
                html_text = get_html_text(study_url)
            
            if html_text:
                keywords = extract_keywords(html_text, top_n=5)
            else:
                keywords = []
            keywords_list.append(keywords)
            continue
        
        # 4) Extraction du texte depuis le PDF
        text = extract_text_from_pdf(pdf_content)
        if text:
            keywords = extract_keywords(text, top_n=5)
        else:
            # Si le PDF est vide ou inexploitable, fallback HTML
            if use_abstract:
                html_text = extract_html_section_text(study_url, container_id="eng-abstract")
                if not html_text:
                    html_text = get_html_text(study_url)
            else:
                html_text = get_html_text(study_url)
            if html_text:
                keywords = extract_keywords(html_text, top_n=5)
            else:
                keywords = []
        
        keywords_list.append(keywords)
    
    df['key_word'] = keywords_list
    return df


In [28]:
data_1_processed = process_dataframe(data_1, use_abstract=True)




Traitement de l'article 0 : https://www.ncbi.nlm.nih.gov/pubmed/32472191/
Aucun lien PDF trouvé sur la page: https://www.ncbi.nlm.nih.gov/pubmed/32472191/
Aucun lien PDF trouvé, on va extraire le texte HTML.
Traitement de l'article 1 : https://www.ncbi.nlm.nih.gov/pubmed/32463545/
Aucun lien PDF trouvé sur la page: https://www.ncbi.nlm.nih.gov/pubmed/32463545/
Aucun lien PDF trouvé, on va extraire le texte HTML.
Traitement de l'article 2 : http://medrxiv.org/cgi/content/short/2020.05.25.20111757v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.25.20111757v1.full.pdf
Traitement de l'article 3 : http://medrxiv.org/cgi/content/short/2020.05.25.20111757v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.25.20111757v1.full.pdf
Traitement de l'article 4 : http://medrxiv.org/cgi/content/short/2020.05.25.20111757v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.25.20111757v1.full.pdf
Traitement de l'article 5 : http://medrxiv.org/cgi/conte

In [ ]:
data_2_processed = process_dataframe(data_2, use_abstract=True)

Traitement de l'article 0 : https://doi.org/10.1016/j.jcv.2020.104371
Aucun lien PDF trouvé sur la page: https://doi.org/10.1016/j.jcv.2020.104371
Aucun lien PDF trouvé, on va extraire le texte HTML.
Traitement de l'article 1 : https://doi.org/10.1016/j.jcv.2020.104364
Aucun lien PDF trouvé sur la page: https://doi.org/10.1016/j.jcv.2020.104364
Aucun lien PDF trouvé, on va extraire le texte HTML.
Traitement de l'article 2 : https://doi.org/10.1111/dom.14105
Erreur lors du téléchargement de la page: https://doi.org/10.1111/dom.14105 - Code 403
Aucun lien PDF trouvé, on va extraire le texte HTML.
Erreur lors de la requête: https://doi.org/10.1111/dom.14105 - Code 403
Erreur lors du téléchargement de la page: https://doi.org/10.1111/dom.14105 - Code 403
Traitement de l'article 3 : https://doi.org/10.1111/dom.14105
Erreur lors du téléchargement de la page: https://doi.org/10.1111/dom.14105 - Code 403
Aucun lien PDF trouvé, on va extraire le texte HTML.
Erreur lors de la requête: https://do

In [ ]:
data_3_processed = process_dataframe(data_3, use_abstract=True)

Traitement de l'article 0 : http://medrxiv.org/cgi/content/short/2020.05.29.20086645v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.29.20086645v1.full.pdf
Traitement de l'article 1 : https://www.ncbi.nlm.nih.gov/pubmed/32463545/
Aucun lien PDF trouvé sur la page: https://www.ncbi.nlm.nih.gov/pubmed/32463545/
Aucun lien PDF trouvé, on va extraire le texte HTML.
Traitement de l'article 2 : http://medrxiv.org/cgi/content/short/2020.05.24.20111823v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.24.20111823v1.full.pdf
Traitement de l'article 3 : http://medrxiv.org/cgi/content/short/2020.05.24.20111823v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.24.20111823v1.full.pdf
Traitement de l'article 4 : http://medrxiv.org/cgi/content/short/2020.05.24.20111724v1?rss=1
Lien PDF trouvé: http://medrxiv.org/content/10.1101/2020.05.24.20111724v1.full.pdf
Traitement de l'article 5 : http://medrxiv.org/cgi/content/short/2020.05.24.20111724v1?r

## Garder les colonnes importantes

In [ ]:

data_1_processed = data_1_processed[["date", "study", "study_link", "journal", "severe", "severe_lower_bound", "severe_upper_bound", "severe_p-value", "severe_significant", "fatality_significant", "fatality_adjusted", "fatality_calculated", "multivariate_adjustment", "study_type", "sample_size", "study_population", "critical_only", "discharged_vs._death?"]]
data_2_processed = data_2_processed[["date", "study", "study_link", "journal", "severe", "severe_lower_bound", "severe_upper_bound", "severe_p-value", "severe_significant", "fatality_significant", "fatality_adjusted", "fatality_calculated", "multivariate_adjustment", "study_type", "sample_size", "study_population", "critical_only", "discharged_vs._death?"]]
data_3_processed = data_3_processed[["date", "study", "study_link", "journal", "severe", "severe_lower_bound", "severe_upper_bound", "severe_p-value", "severe_significant", "fatality_significant", "fatality_adjusted", "fatality_calculated", "multivariate_adjustment", "study_type", "sample_size", "study_population", "critical_only", "discharged_vs._death?"]]


## Création de la colonnes "context"

In [ ]:
# Création d'une colonne 'context' dans data_1_processed, data_2_processed ,data_3_processed
# Cette colonne regroupe plusieurs informations clés de l'étude sous forme de texte lisible.
data_1_processed["context"] = (
    "Date: " + data_1_processed["date"].astype(str) + ". " +
    "Study: " + data_1_processed["study"] + ". " +
    "Study Link: " + data_1_processed["study_link"] + ". " +
    "Journal: " + data_1_processed["journal"] + ". " +
    "Severe: " + data_1_processed["severe"].astype(str) + ". " +
    "Severe Lower Bound: " + data_1_processed["severe_lower_bound"].astype(str) + ". " +
    "Severe Upper Bound: " + data_1_processed["severe_upper_bound"].astype(str) + ". " +
    "Severe p-value: " + data_1_processed["severe_p-value"].astype(str) + ". " +
    "Severe Significant: " + data_1_processed["severe_significant"].astype(str) + ". " +
    "Fatality Significant: " + data_1_processed["fatality_significant"].astype(str) + ". " +
    "Fatality Adjusted: " + data_1_processed["fatality_adjusted"].astype(str) + ". " +
    "Fatality Calculated: " + data_1_processed["fatality_calculated"].astype(str) + ". " +
    "Multivariate Adjustment: " + data_1_processed["multivariate_adjustment"].astype(str) + ". " +
    "Study Type: " + data_1_processed["study_type"] + ". " +
    "Sample Size: " + data_1_processed["sample_size"].astype(str) + ". " +
    "Study Population: " + data_1_processed["study_population"] + ". " +
    "Critical Only: " + data_1_processed["critical_only"] + ". " +
    "Discharged vs Death: " + data_1_processed["discharged_vs._death?"] + "."
)


data_2_processed["context"] = (
    "Date: " + data_2_processed["date"].astype(str) + ". " +
    "Study: " + data_2_processed["study"] + ". " +
    "Study Link: " + data_2_processed["study_link"] + ". " +
    "Journal: " + data_2_processed["journal"] + ". " +
    "Severe: " + data_2_processed["severe"].astype(str) + ". " +
    "Severe Lower Bound: " + data_2_processed["severe_lower_bound"].astype(str) + ". " +
    "Severe Upper Bound: " + data_2_processed["severe_upper_bound"].astype(str) + ". " +
    "Severe p-value: " + data_2_processed["severe_p-value"].astype(str) + ". " +
    "Severe Significant: " + data_2_processed["severe_significant"].astype(str) + ". " +
    "Fatality Significant: " + data_2_processed["fatality_significant"].astype(str) + ". " +
    "Fatality Adjusted: " + data_2_processed["fatality_adjusted"].astype(str) + ". " +
    "Fatality Calculated: " + data_2_processed["fatality_calculated"].astype(str) + ". " +
    "Multivariate Adjustment: " + data_2_processed["multivariate_adjustment"].astype(str) + ". " +
    "Study Type: " + data_2_processed["study_type"] + ". " +
    "Sample Size: " + data_2_processed["sample_size"].astype(str) + ". " +
    "Study Population: " + data_2_processed["study_population"] + ". " +
    "Critical Only: " + data_2_processed["critical_only"] + ". " +
    "Discharged vs Death: " + data_2_processed["discharged_vs._death?"] + "."
)
data_3_processed["context"] = (
    "Date: " + data_3_processed["date"].astype(str) + ". " +
    "Study: " + data_3_processed["study"] + ". " +
    "Study Link: " + data_3_processed["study_link"] + ". " +
    "Journal: " + data_3_processed["journal"] + ". " +
    "Severe: " + data_3_processed["severe"].astype(str) + ". " +
    "Severe Lower Bound: " + data_3_processed["severe_lower_bound"].astype(str) + ". " +
    "Severe Upper Bound: " + data_3_processed["severe_upper_bound"].astype(str) + ". " +
    "Severe p-value: " + data_3_processed["severe_p-value"].astype(str) + ". " +
    "Severe Significant: " + data_3_processed["severe_significant"].astype(str) + ". " +
    "Fatality Significant: " + data_3_processed["fatality_significant"].astype(str) + ". " +
    "Fatality Adjusted: " + data_3_processed["fatality_adjusted"].astype(str) + ". " +
    "Fatality Calculated: " + data_3_processed["fatality_calculated"].astype(str) + ". " +
    "Multivariate Adjustment: " + data_3_processed["multivariate_adjustment"].astype(str) + ". " +
    "Study Type: " + data_3_processed["study_type"] + ". " +
    "Sample Size: " + data_3_processed["sample_size"].astype(str) + ". " +
    "Study Population: " + data_3_processed["study_population"] + ". " +
    "Critical Only: " + data_3_processed["critical_only"] + ". " +
    "Discharged vs Death: " + data_3_processed["discharged_vs._death?"] + "."
)


/tmp/ipykernel_255311/141344596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_processed["context"] = (
/tmp/ipykernel_255311/141344596.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2_processed["context"] = (
/tmp/ipykernel_255311/141344596.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

## Enregistrement des résultats du preprocessing en .CSV

In [ ]:

data_1_processed.to_csv("/home/cytech/Rag_covid/data/data_preprocessing/Age_Preprocessing.csv", index=False)

data_2_processed.to_csv("/home/cytech/Rag_covid/data/data_preprocessing/Diabetes_Preprocessing.csv", index=False)

data_3_processed.to_csv("/home/cytech/Rag_covid/data/data_preprocessing/Overweight_or_obese_Preprocessing.csv", index=False)